In [1]:
!pip install -q transformers accelerate bitsandbytes jinja2 beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.2 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [3]:
# 📦 Install dependencies
!pip install beautifulsoup4 jinja2 requests

# 📁 Upload required files
from google.colab import files
print("📤 Upload input_ui.html, policy.json, and dashboard_template.html")
#uploaded = files.upload()

📤 Upload input_ui.html, policy.json, and dashboard_template.html


In [4]:
# 🧠 Hugging Face API setup

from huggingface_hub import login
from google.colab import userdata


hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)
#API_URL = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"

#headers = {"Authorization": f"Bearer {HF_TOKEN}"}

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load quantized model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)

# Create pipeline
llm = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [6]:
# 📄 Load policy
import json
with open("policy.json", "r", encoding="utf-8") as f:
    policy = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'policy.json'

In [ ]:
# 🔍 Extract UI elements
from bs4 import BeautifulSoup

def extract_ui_elements(html_path):
    with open(html_path, 'r', encoding='utf-8') as f:
        html = f.read()
        soup = BeautifulSoup(html, 'html.parser')

    elements = []
    for tag in soup.find_all(['label', 'input', 'button']):
        text = tag.get_text(strip=True)
        if text:
            line_number = html[:html.find(str(tag))].count('\n') + 1
            parent = tag.find_parent()
            parent_name = parent.name if parent else None
            elements.append({
                'element_type': tag.name,
                'element_text': text
            })
    for option in soup.find_all('option'):
        option_text = option.get_text(strip=True)
        if option_text:
            line_number = html[:html.find(str(option))].count('\n') + 1
            parent = option.find_parent()
            parent_name = parent.name if parent else None
            elements.append({
                'element_type': 'option',
                'element_text': option_text
            })
    return elements

In [ ]:
def validate_with_llm(element, policy):
    prompt = f"""
You are a UI compliance auditor. For the following UI element, check if its text contains any prohibited terms from this list: {policy['prohibited_terms']}.
Also check if it contains any required phrases from this list: {policy['required_phrases']}.
Also check accessibility guidelines.

Element Type: {element['element_type']}
Element Text: {element['element_text']}

Policy:
- Prohibited terms: {policy['prohibited_terms']}
- Required phrases: {policy['required_phrases']}
- Accessibility guidelines: {policy['accessibility_guidelines']}

Respond with:
- Compliance Status: Compliant / Needs Review / Non-Compliant
- Reason (always specify why if Non-Compliant, including which prohibited term or required phrase is missing)
- Suggested Correction (if any)
"""
    try:
        result = pipeline(prompt, max_new_tokens=200, do_sample=True, temperature=0.7)[0]["generated_text"]
        return result
    except Exception as e:
        print("LLM error:", e)
        return ""

In [ ]:
import re

In [ ]:
# 🧠 Parse LLM response


def parse_llm_response(response):
    status_match = re.search(r"Compliance Status:\s*(.*)", response)
    reason_match = re.search(r"Reason:\s*(.*)", response)
    suggestion_match = re.search(r"Suggested Correction:\s*(.*)", response)

    status = status_match.group(1).strip() if status_match else "Unknown"
    reason = reason_match.group(1).strip() if reason_match else ""
    suggestion = suggestion_match.group(1).strip() if suggestion_match else ""

    status_class = {
        "Compliant": "compliant",
        "Non-Compliant": "non-compliant",
        "Needs Review": "needs-review"
    }.get(status, "")

    return status, reason, suggestion, status_class


In [ ]:
# 🖥️ Generate dashboard
from jinja2 import Environment, FileSystemLoader

def generate_dashboard(results, output_path):
    env = Environment(loader=FileSystemLoader('.'))
    template = env.get_template('dashboard_template.html')
    html = template.render(results=results)

    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html)

In [ ]:
# 🚀 Run the pipeline
elements = extract_ui_elements("input_ui.html")
results = []

for element in elements:
    response = validate_with_llm(element, policy)
    status, reason, suggestion, status_class = parse_llm_response(response)
    results.append({
        "element_type": element["element_type"],
        "element_text": element["element_text"],
        "status": status,
        "reason": reason,
        "suggestion": suggestion,
        "status_class": status_class
    })

generate_dashboard(results, "output_report.html")
print("✅ Compliance report generated: output_report.html")

In [ ]:
test_element = {
    "element_type": "label",
    "element_text": "Email Address",
    "line_number": 12,
    "parent": "form"
}

response = validate_with_llm(test_element, policy)
print("🔍 Raw LLM Output:\n", response)

status, reason, suggestion, status_class = parse_llm_response(response)
print("\n✅ Parsed Output:")
print("Status:", status)
print("Reason:", reason)
print("Suggestion:", suggestion)